In [1]:
# ライブラリのimport
import subprocess
import platform
import pandas as pd

In [2]:
# 設定変数のimport
from config import OS, APPS, BASE_PORT, ALL_PORT_RANGE
from os_utils import run_command, get_port_search_command, get_process_details

In [3]:
# OSの確認
OS

'Darwin'

# 起動プロセスの取得

In [4]:
# 調べるポートを設定。実際はforで供給される

# アプリ起動済みport
# 事前にstreamlit run {app} --server.port=8501などで起動しておく
running_port = 8501

# 起動していないport
not_running_port = 8510

In [5]:
# OSに依存せずportを調べる処理
search_command = get_port_search_command(port=running_port)
search_output = run_command(search_command)

def shape_to_dataframe(search_output: str) -> pd.DataFrame:
    # ヘッダー: 1次元のリスト
    header = search_output.split("\n")[0].split()
    # 要素: 2次元のリスト
    lines = list(map(
        lambda line: line.split(maxsplit=len(header)-1),
        search_output.split("\n")[1:]
    ))
    # dfにして返す
    return pd.DataFrame(
        lines,
        columns=header
    )

port_search_result = shape_to_dataframe(search_output)

In [6]:
# ユニークな要素をカンマ区切りでまとめる関数
def summarize_unique_values(df) -> pd.DataFrame:
    merged_data = {}
    for col in df.columns:
        unique_values = set(df[col])  # ユニークな値を取得
        unique_values.discard(None)  # None（欠損値）は除外
        merged_data[col] = ", ".join(unique_values)  # カンマ区切りで結合
    return pd.DataFrame([merged_data])  # 1行のDataFrameとして返す

slim_result = summarize_unique_values(port_search_result)

In [7]:
def extract_app_name(process_output: str) -> str:
    """
    プロセスの詳細情報からアプリ名を抽出
    """
    # Google Chrome
    if "Google Chrome" in process_output:
        return "Google Chrome"
    # Safari
    if "Safari" in process_output:
        return "Safari"
    # Streamlit アプリ
    if "streamlit run" in process_output:
        for app, info in APPS.items():
            if info["path"] in process_output:
                return app
        return "Streamlit App"  # どのアプリにも一致しない場合

    return "Unknown"

def add_extracted_app_name(df):
    """DataFrame にアプリ名情報を追加"""
    if "PID" not in df.columns:
        return df  # PID カラムがない場合はそのまま返す

    pids = df["PID"].iloc[0].split(", ")  # PID をリスト化
    app_names = [extract_app_name(get_process_details(pid.strip())) for pid in pids if pid.strip().isdigit()]

    df["APP_NAME"] = ", ".join(set(app_names)) if app_names else "Unknown"
    return df

port_result = add_extracted_app_name(slim_result)

In [8]:
# 結果を出力
print("-- search_output --")
print(search_output)
print("\n-- processed --")
# 以下はメソッドに切り出したのでコメントアウト
# print("header size:", len(header))
# print(header)
print("\n-- converted to dataframe --")
print(port_search_result)
print("\n-- omit duplicates --")
print(slim_result)
print("\n-- details --")
print(port_result)


-- search_output --


-- processed --

-- converted to dataframe --
Empty DataFrame
Columns: []
Index: []

-- omit duplicates --
Empty DataFrame
Columns: []
Index: [0]

-- details --
Empty DataFrame
Columns: []
Index: [0]


In [9]:
slim_result

""
0


In [10]:
port_result

""
0
